In [1]:
get_ipython().run_line_magic('pylab', 'inline')

Populating the interactive namespace from numpy and matplotlib


In [2]:
#from tinc import *
import tinc
import os
import numpy as np

In [3]:
#Enlarge the jupyter notebook so that plots are easier to see 
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [4]:
A = tinc.Parameter("A", "weighting")
A.values = np.linspace(.1,2,3)

B = tinc.Parameter("B", "weighting")
B.values = np.linspace(0,2,3)

kT = tinc.Parameter("kT", "weighting")
kT.values = np.linspace(.01,.1,3)


graph_buffer = tinc.DiskBufferImage("graph", "out.png", "graph_output")

In [5]:
ps = tinc.ParameterSpace("ps")
ps.register_parameters([A, B, kT])
ps.enable_cache()

Validating json with /home/derick/anaconda3/envs/casm_1.2.0/lib/python3.9/site-packages/tinc/tinc_cache_schema.json


In [6]:
def make_graph(A, B, kT):
    
    #hard-coding this now, change later
    ce_fits_dir = '/media/derick/DeoResearch/experiments/ZrN_FCC_1.2.0_demo/tinc_cluster_expansion_fits'
    prefix = 'ZrN_FCC_1.2.0_demo_'
    
    fit_dir = 'A_%s_B_%s_kt_%s' % ((A), (B), (kT))

    
    image_name = prefix + fit_dir + '.png'
    #figure(figsize=[18.5,10])
    #close() # Avoid showing as an additional graph in jupyter
    
    #Path to pre-made images, using scripts from vasp_scripts
    image_path = os.path.join(ce_fits_dir, fit_dir, image_name)
    file = open(image_path, 'rb')
    return file.read()


def value_changed(value):
    imagedata = ps.run_process(make_graph)
    graph_buffer.data = imagedata

A.register_callback(value_changed)
B.register_callback(value_changed)
kT.register_callback(value_changed)
#kT.register_callback(cb)

In [7]:
print(A.values,B.values,kT.values)

[0.1  1.05 2.  ] [0. 1. 2.] [0.01  0.055 0.1  ]


#currently not using this display approach
from ipywidgets import GridspecLayout
grid = GridspecLayout(3, 2, height='300px')
grid[:, 1] = graph_buffer.interactive_widget()
grid[0, 0] = A.interactive_widget()
grid[1, 0] = B.interactive_widget()
grid[2, 0] = kT.interactive_widget()
grid


In [9]:
from ipywidgets import Button, HBox, VBox

box = VBox([A.interactive_control(), 
            B.interactive_control(), 
            kT.interactive_control(), 
            graph_buffer.interactive_widget()])
graph_buffer._interactive_widget.width = "90%"
box